In [22]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/master.csv"
df = pd.read_csv(filepath)
df

,page_id,page_title,view_count,protection_status,page_length,edit_count,page_watchers,page_watchers_recent_edits,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
0,11291,Floccinaucinihilipilification,160037,autoconfirmed,373.00,709.00,69,8,484.00,11291.00,12.00,0.00,0.00,3.00
1,1143923,Honolulu University,41519,autoconfirmed,3365.00,165.00,Fewer than 30 watchers,NaN,461.00,1143923.00,1.00,0.00,0.00,3.00
2,1001251,Robert Spencer (author),532302,autoconfirmed,65.00,2.00,210,41,1825.00,60596940.00,0.00,0.00,0.00,3.00
3,1126490,Yard of ale,590917,autoconfirmed,5300.00,352.00,41,4,5641.00,1126490.00,6.00,0.00,0.00,3.00
4,1033826,Professional wrestling attacks,2967525,autoconfirmed,91.00,1.00,376,7,6476.00,62814991.00,0.00,0.00,0.00,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,18342008,"Carlos, Duke of Parma",241878,unprotected,59.00,1.00,Fewer than 30 watchers,NaN,2424.00,52782565.00,0.00,0.00,0.00,nan
829,40792858,2013–14 MŠK Žilina season,3325,unprotected,4803.00,11.00,Fewer than 30 watchers,NaN,9.00,40792858.00,3.00,1.00,1.00,nan
830,14531947,Politics of Anhui,44043,unprotected,3579.00,64.00,Fewer than 30 watchers,NaN,35.00,14531947.00,3.00,2.00,1.00,nan
831,5482190,Dan Sikes,28302,unprotected,13221.00,123.00,Fewer than 30 watchers,NaN,182.00,5482190.00,0.00,1.00,1.00,nan


In [23]:
# correlation matrix 

df.corr()

,page_id,view_count,page_length,edit_count,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
page_id,1.00,-0.00,-0.31,-0.33,-0.00,0.86,-0.27,-0.13,-0.22,-0.35
view_count,-0.00,1.00,-0.02,0.02,1.00,-0.00,0.06,-0.01,-0.01,0.32
page_length,-0.31,-0.02,1.00,0.81,-0.01,-0.34,0.56,0.47,0.59,0.73
edit_count,-0.33,0.02,0.81,1.00,0.02,-0.36,0.60,0.50,0.64,0.69
page_views_past_30days,-0.00,1.00,-0.01,0.02,1.00,-0.01,0.06,-0.01,-0.01,0.52
page_id_scrapped,0.86,-0.00,-0.34,-0.36,-0.01,1.00,-0.29,-0.15,-0.25,-0.38
redirects,-0.27,0.06,0.56,0.60,0.06,-0.29,1.00,0.29,0.39,0.42
recent_number_of_edits,-0.13,-0.01,0.47,0.50,-0.01,-0.15,0.29,1.00,0.77,0.33
number_distinct_authors,-0.22,-0.01,0.59,0.64,-0.01,-0.25,0.39,0.77,1.00,0.48
number_categories,-0.35,0.32,0.73,0.69,0.52,-0.38,0.42,0.33,0.48,1.00


In [3]:
features = df

# Labels are the values we want to predict
labels = np.array(df['protection_status'])

# Remove the labels from the features
drop_column_list = ['protection_status', 
                    'page_title', 
                    'page_id', 
                    'page_id_scrapped', 
                    'page_length', 
                    'page_watchers_recent_edits', 
                    'redirects', 
                    'recent_number_of_edits'
                   ]
features = features.drop(drop_column_list, axis =1)

features

,view_count,edit_count,page_watchers,page_views_past_30days,number_distinct_authors,number_categories
0,160037,709.0,69,484.0,0.0,3.0
1,41519,165.0,Fewer than 30 watchers,461.0,0.0,3.0
2,532302,2.0,210,1825.0,0.0,3.0
3,590917,352.0,41,5641.0,0.0,3.0
4,2967525,1.0,376,6476.0,0.0,3.0
...,...,...,...,...,...,...
828,241878,1.0,Fewer than 30 watchers,2424.0,0.0,NaN
829,3325,11.0,Fewer than 30 watchers,9.0,1.0,NaN
830,44043,64.0,Fewer than 30 watchers,35.0,1.0,NaN
831,28302,123.0,Fewer than 30 watchers,182.0,1.0,NaN


In [4]:
#Encode string to float
features.loc[features['page_watchers'] == "Fewer than 30 watchers", 'page_watchers'] = 25

#Convert String to Floats
features['edit_count'] = features['edit_count'].astype(float)
features['page_watchers'] = features['page_watchers'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)


In [14]:
# kNN Imputer

from sklearn.impute import KNNImputer
imputer = KNNImputer (n_neighbors = 5)

features = imputer.fit_transform(features)

In [15]:
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)   
        
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.10, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.03it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
RandomForestClassifier             0.68               0.68    None      0.67   
LGBMClassifier                     0.64               0.65    None      0.64   
ExtraTreesClassifier               0.64               0.65    None      0.64   
CalibratedClassifierCV             0.62               0.63    None      0.60   
BaggingClassifier                  0.62               0.62    None      0.61   
XGBClassifier                      0.62               0.62    None      0.61   
LinearSVC                          0.61               0.61    None      0.59   
LogisticRegression                 0.60               0.60    None      0.58   
ExtraTreeClassifier                0.58               0.59    None      0.56   
SVC                                0.57               0.58    None      0.55   
KNeighborsClassifier               0.57 

# Apply scaling

In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

features = scaler.fit_transform(features)

In [19]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.10, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.23it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
RandomForestClassifier             0.68               0.68    None      0.67   
LGBMClassifier                     0.64               0.65    None      0.64   
ExtraTreesClassifier               0.64               0.65    None      0.64   
CalibratedClassifierCV             0.62               0.63    None      0.60   
BaggingClassifier                  0.62               0.62    None      0.61   
XGBClassifier                      0.62               0.62    None      0.61   
LinearSVC                          0.61               0.61    None      0.59   
LogisticRegression                 0.60               0.60    None      0.58   
ExtraTreeClassifier                0.58               0.59    None      0.56   
SVC                                0.57               0.58    None      0.55   
KNeighborsClassifier               0.57 

In [20]:
from sklearn.preprocessing import StandardScaler
scaler_standard = StandardScaler()

features = scaler.fit_transform(features)

In [21]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.10, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.11it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
RandomForestClassifier             0.68               0.68    None      0.67   
LGBMClassifier                     0.64               0.65    None      0.64   
ExtraTreesClassifier               0.64               0.65    None      0.64   
CalibratedClassifierCV             0.62               0.63    None      0.60   
BaggingClassifier                  0.62               0.62    None      0.61   
XGBClassifier                      0.62               0.62    None      0.61   
LinearSVC                          0.61               0.61    None      0.59   
LogisticRegression                 0.60               0.60    None      0.58   
ExtraTreeClassifier                0.58               0.59    None      0.56   
SVC                                0.57               0.58    None      0.55   
KNeighborsClassifier               0.57 

In [13]:
features

array([[4.11376103e-06, 1.93739054e-02, 3.76631714e-04, 2.91998745e-06,
        0.00000000e+00, 1.31578947e-02],
       [1.06722944e-06, 4.48774081e-03, 0.00000000e+00, 2.78122772e-06,
        0.00000000e+00, 1.31578947e-02],
       [1.36829158e-05, 2.73642732e-05, 1.58356516e-03, 1.10102833e-05,
        0.00000000e+00, 1.31578947e-02],
       [1.51896274e-05, 9.60485989e-03, 1.36956987e-04, 3.40323331e-05,
        0.00000000e+00, 1.31578947e-02],
       [7.62808655e-05, 0.00000000e+00, 3.00449390e-03, 3.90699147e-05,
        0.00000000e+00, 1.31578947e-02],
       [2.29010400e-06, 0.00000000e+00, 4.27990584e-05, 1.25064752e-05,
        1.53846154e-02, 1.31578947e-02],
       [1.72201601e-05, 4.95293345e-02, 9.41579285e-05, 6.44388140e-05,
        3.07692308e-02, 1.31578947e-02],
       [2.00680674e-07, 1.14929947e-03, 0.00000000e+00, 1.38759734e-07,
        0.00000000e+00, 2.63157895e-02],
       [1.01828669e-06, 1.69932137e-02, 9.41579285e-05, 1.00148330e-06,
        0.00000000e+00, 

In [24]:
features

array([[4.11376103e-06, 1.93739054e-02, 3.76631714e-04, 2.91998745e-06,
        0.00000000e+00, 1.31578947e-02],
       [1.06722944e-06, 4.48774081e-03, 0.00000000e+00, 2.78122772e-06,
        0.00000000e+00, 1.31578947e-02],
       [1.36829158e-05, 2.73642732e-05, 1.58356516e-03, 1.10102833e-05,
        0.00000000e+00, 1.31578947e-02],
       [1.51896274e-05, 9.60485989e-03, 1.36956987e-04, 3.40323331e-05,
        0.00000000e+00, 1.31578947e-02],
       [7.62808655e-05, 0.00000000e+00, 3.00449390e-03, 3.90699147e-05,
        0.00000000e+00, 1.31578947e-02],
       [2.29010400e-06, 0.00000000e+00, 4.27990584e-05, 1.25064752e-05,
        1.53846154e-02, 1.31578947e-02],
       [1.72201601e-05, 4.95293345e-02, 9.41579285e-05, 6.44388140e-05,
        3.07692308e-02, 1.31578947e-02],
       [2.00680674e-07, 1.14929947e-03, 0.00000000e+00, 1.38759734e-07,
        0.00000000e+00, 2.63157895e-02],
       [1.01828669e-06, 1.69932137e-02, 9.41579285e-05, 1.00148330e-06,
        0.00000000e+00, 